In [ ]:
import pandas as pd   # pyright: ignore[reportMissingImports]
import numpy as np  # pyright: ignore[reportMissingImports]
import re  
from sklearn.utils import shuffle  # pyright: ignore[reportMissingImports]

In [ ]:
df_human = pd.read_csv(r"C:\Users\Eslem\OneDrive\Masaüstü\HumanOrAI\data\raw\human_abstracts.csv")
df_ai = pd.read_csv(r"C:\Users\Eslem\OneDrive\Masaüstü\HumanOrAI\data\raw\ai_abstracts.csv")
print("dosyalar yuklendi")
print(f"human veri sayısı: {len(df_human)}")
print(f"ai veri sayısı: {len(df_ai)}")

dosyalar yuklendi
human veri sayısı: 3005
ai veri sayısı: 3001


In [ ]:
#label boşluk kontrol
bos_human = df_human['label'].isnull().sum()
if bos_human> 0:
    df_human['label'] = df_human['label'].fillna("Human")

bos_ai = df_ai['label'].isnull().sum()
if bos_ai> 0:
    df_ai['label'] = df_ai['label'].fillna("Ai")


In [ ]:
#human verisi temizlik
def clean_human(text):
    if not isinstance(text, str): return ""
    text =re.sub(r'\$.*?\$','',text)
    text =re.sub(r'\[d+\]','',text)
    text =re.sub(r'^abstract\s*[:\.]?','',text,flags=re.IGNORECASE) 
    return text

In [ ]:
#ai verisi temizleme
def clean_ai(text):
    if not isinstance(text, str): return ""
    text = re.sub(r"^\s*with different sentence structures\**\s*", "", text, flags=re.IGNORECASE)
    patterns = [r"^here is a summary.*?:",r"^sure, i can (help|provide).*",
                 r"^certainly.*?",r"^in this (study|paper),",
                 r"^summary:",r"^abstract:",
                 r"generated by ai",r"ai-generated content",]
    for p in patterns:
        text = re.sub(p, '', text, flags=re.IGNORECASE)
        
    text = re.sub(r'[\s\-*]+$', '', text)
    
    return text.strip()

In [ ]:
df_ai["abstract_text"] = df_ai["abstract_text"].apply(clean_ai)
df_ai["abstract_text"].head(3)

In [ ]:
#veri setlerini birleştirme
df_master=pd.concat([df_human, df_ai], ignore_index=True)


In [ ]:
#temizlik
def general_cleaning(text):
    if not isinstance(text,str): return""
    text =text.lower()
    text = re.sub(r'http\S+', '', text)
    text =re.sub(r'\n', ' ', text)
    text =re.sub(r'\s+', ' ', text).strip()
    return text

df_master['cleaned_text']=df_master['abstract_text'].apply(general_cleaning)

In [ ]:
df_master=df_master[df_master['cleaned_text'].str.len()>10]

df_master.drop_duplicates(subset=['cleaned_text'],inplace=True)

df_master=shuffle(df_master, random_state=42).reset_index(drop=True)


In [ ]:
dagilim = df_master['label'].value_counts()
print(f"Toplam Temiz Veri Sayısı: {len(df_master)}")
print(dagilim)

Toplam Temiz Veri Sayısı: 6000
label
Human    3000
AI       3000
Name: count, dtype: int64


In [ ]:
df_master.to_csv(r"C:\Users\Eslem\OneDrive\Masaüstü\HumanOrAI\data\raw\clean_dataset.csv")